# 重み付き有向グラフを用いたマウス脳内の投射関係解析

マウス脳の注射実験データを統合して、各脳部位間の重み付き有向グラフを作成しました。

重みには"normalized-projection-volume"をはじめ、各パラメータの平均値を記録しました。

In [ ]:
# Pythonライブラリインストール
# ※Python 3.10.x　使用推奨
!python --version
!pip install python-dotenv
!pip install --upgrade openai
!pip install openai[datalib]

!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install plotly
!pip install scikit-learn
!pip install sqlalchemy

## 環境変数
supabase接続用URL,APIキーと、openai api接続用のAPIキーを設定します。
自身のopenaiアカウントからapi keyを取得してください。

https://platform.openai.com/account/api-keys

supabaseの情報は管理者にお尋ねください。

下記の例では、.envファイルに変数を書き込んで、JupiterNotebookで読み込む仕様で実装しております。

※.envファイルの作成が困難、.envファイルから値を読み込めない場合、
　os.getenv("◯◯")部分に変数値を直接書き込んでいただいても動作自体には問題ありません。

In [65]:
# 環境変数
import os
from dotenv import load_dotenv

import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import text
import urllib.parse
from IPython.display import display
import openai

load_dotenv()

# supabase接続用変数
db_host = os.getenv("DB_HOST")
db_port = os.getenv("DB_PORT")
db_name = os.getenv("DB_NAME")
db_user = os.getenv("DB_USER")
db_pass = os.getenv("DB_PASS")

# OPENAI API KEY
openai_api_key = os.getenv("OPENAI_API_KEY")
openai.api_key = openai_api_key

# Connect to the database
connection_config = {
    'user': db_user,
    'password': urllib.parse.quote_plus(db_pass),
    'host': db_host,
    'port': db_port, 
    'database': db_name
}
engine = create_engine('postgresql://{user}:{password}@{host}:{port}/{database}'.format(**connection_config))


print('環境変数読み込み完了')

環境変数読み込み完了


# 処理実行
## 重み付き隣接行列（有向グラフ）の作成

重みの閾値、有向グラフのノード数の上限値を設定します。

脳構造名または頭字語でのラベリング、重み付けを整数値, 浮動小数点で行うなど様々なパターンでのグラフを生成しています。
（次のステップでどのグラフをpromptに付与するかを設定します。）

In [ ]:
# 重みに採用するパラメータ。下記から選択してください。
# normalized-projection-volume, projection-density, projection-energy, projection-intensity, projection-volume
column_name = "normalized-projection-volume"
# 重みの閾値。閾値以下のデータは参照されません。
threshold_value = 0.1
# 有向グラフのノード数の上限値
threshold_rank = 5

sql =f"""
SELECT
    pg."{column_name}",
    pg."injected-structure-id",
    pg."projected-structure-id",
    si.name AS "injected-structure-name",
    si.acronym AS "injected-structure-acronym",
    sp.name AS "projected-structure-name",
    sp.acronym AS "projected-structure-acronym"
FROM projection_graphs AS pg
    INNER JOIN structures AS si ON pg."injected-structure-id" = si.id
    INNER JOIN structures AS sp ON pg."projected-structure-id" = sp.id
WHERE
    pg."{column_name}" > {threshold_value}
    AND si."st-level" > 5
    AND sp."st-level" > 5
ORDER BY 
    pg."{column_name}" DESC
"""
with engine.begin() as conn:
    query = text(sql)
    df = pd.read_sql_query(query, conn)
    display(df)


# 重み付き隣接行列をJSONとして書き出し
acronym_edges = []
name_edges = []
acronym_matrix = {}
acronym_matrix_int = {}
acronym_matrix_float = {}
name_matrix = {}
acronym_dict = {}
for index, row in df.iterrows():
    acronym_dict[row["projected-structure-acronym"]] = row["projected-structure-name"]
    acronym_dict[row["injected-structure-acronym"]] = row["injected-structure-name"]
    # Check if the key is not in the matrix
    if row["injected-structure-acronym"] not in acronym_matrix:
        acronym_matrix[row["injected-structure-acronym"]] = []
        acronym_matrix_int[row["injected-structure-acronym"]] = {}
        acronym_matrix_float[row["injected-structure-acronym"]] = {}
        name_matrix[row["injected-structure-name"]] = {}

    # Add the value to the nested dictionary
    if len(acronym_matrix[row["injected-structure-acronym"]]) < threshold_rank: 
        acronym_matrix[row["injected-structure-acronym"]].append(row["projected-structure-acronym"])
        acronym_matrix_int[row["injected-structure-acronym"]][row["projected-structure-acronym"]]=int(row[column_name])
        acronym_matrix_float[row["injected-structure-acronym"]][row["projected-structure-acronym"]]=float(row[column_name])
        name_matrix[row["injected-structure-name"]][row["projected-structure-name"]]=int(row[column_name])
        acronym_edges.append({
            "source": row["injected-structure-acronym"],
            "target": row["projected-structure-acronym"]
        })
        name_edges.append({
            "source": row["injected-structure-name"],
            "target": row["projected-structure-name"]
        })

print(len(acronym_edges))
print(acronym_edges)
print(name_edges)
print(len(acronym_matrix))
print(acronym_matrix)
print(acronym_matrix_int)
print(acronym_matrix_float)
print(name_matrix)

print(len(acronym_dict))
print(acronym_dict)

## OpenAI Completion APIを用いて洞察を得る

教授と学生というロールモデルを用いて、解析情報を報告するというかたちでCompletion APIを使用しました。



In [ ]:
# OpenAI Completion APIを呼び出します。第2引数は有向グラフを送信するかどうか。第3引数は有向グラフに重み情報を付加するかどうかのフラグです。
def request_for_insight(query:str, is_sending_matrix:bool=True, is_Weighted=True):
    print("----")
    print(f"Q. {query}")

    msg=[]
    msg.append({"role": "system", "content": "You are a professor majoring in mouse brain connectivity. And you have a student analyzing experiment data and studying the neural connections in the mouse brain. In this experiment, a viral tracer is injected into a specimen that labels the axons by expressing a fluorescent protein, and the labeled axons are visualized using serial two-photon tomography."})
    if is_sending_matrix:

      if is_Weighted:
        msg.append({"role": "system", "content": f"You have a weighted adjacency matrix (JSON) of each mouse brain structure. It shows {column_name} from injected brain structure to the projected brain structure. \n----\n{acronym_matrix_int}"})
      else:
        msg.append({"role": "system", "content": f"You have an adjacency matrix (JSON) of each mouse brain structure. It shows relationships from injected brain structure to the projected brain structure. \n----\n{acronym_matrix}"})
      
      #msg.append({"role": "system", "content": f"You have a list of edges (JSON) on each mouse brain structure. It shows connectivity from injected brain structure to the projected brain structure. \n----\n{edges}"})
      #msg.append({"role": "system", "content": f"You have a acronym dictionary (JSON). \n----\n{acronym_dict}"})
      
    msg.append({"role": "user", "content": query})
    
    completion = openai.ChatCompletion.create(
      model="gpt-4",
      messages=msg,
      temperature=0.2
    )
    print(f"A. {completion.choices[0].message.content}")
    print("----")
    print(msg)
    print(completion)

queries = [
  "What is the projection relationship from the frontal pole with respect to the mouse brain? Tell me as more details as possible.",
  "What are the main brain regions that project to the frontal pole with respect to the mouse brain? Tell me as more details as possible.",
  "From the point of view of the mouse brain projection connectivity, what is the most similar brain region to the frontal pole? Tell me the reasons as much as you can.",
  ]

for query in queries:
  print("Completion A. 事前情報なし")
  print(request_for_insight(query, False, False))
  print(f"Completion D.事前情報あり 投射量閾値:{threshold_value} 有向ノード閾値:{threshold_rank}")
  request_for_insight(query, True, False)